# Self-Replicating Robot System Configurations

|ID|Design Option|Characteristics|
|-|-|-|
|1|Centralized homogeneous (CHO)|One robot is responsible for both printing components and assembling them. Constructed robots are of the normal type and either gather resources or complete other objectives.|
|2|Decentralized homogeneous (DHO)| All robots have the capability to print components, assemble them, and gather resources or complete other objectives.|
|3|Hierarchical homogeneous (HHO)|There are a variable number of robots capable of printing components and assembling them. There are also a variable number of normal type robots.|
|4|Centralized heterogeneous (CHE)|One robot is responsible for printing components, and another (distinct) robot is responsible for assembling them. Constructed robots are of the normal type and either gather resources or complete other objectives.|
|5|Decentralized heterogeneous (DHE)|Robots have either the capability to print components or the capability to assemble them. All robots can gather resources or complete other objectives.|
|6|Hierarchical heterogeneous (CHE)|There are a variable number of robots capable of printing components, a variable number capable of assembling them (distinct from the printing group), and a variable number of normal type robots. All robots can gather resources or complete other objectives.|



# Libraries 

In [52]:
import random
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import sys
from matplotlib.patches import Rectangle

from _plotly_future_ import v4_subplots
import plotly
import plotly.io as pio
import numpy as np
import chart_studio
import chart_studio.plotly as cspy
import chart_studio.tools as tls

from plotly.subplots import make_subplots
import plotly.graph_objects as go


import plotly.express as px

username = 'vishalgattani' # your username\n",
api_key = 'WSy2EFPTbxYYm3Rmcx53' # your api key - go to profile > settings > regenerate key\n",
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)


# Global Variables

In [53]:
timesteps = 1500

# set random number generator
random.seed()

# global variables
rid = 1
nid = 0
aid = 0
pid = 0
decimalPlaces = 3

# simulation parameters
Num_Steps = 100				#; % Number of iterations/time-steps that the simulation goes through.
NonPr = 300.0 				#; % The robot system=s starting quantity of nonprintable components.
Printable = 100.0 			#; % The robot system’s starting quantity of printable components.
Materials = 50.0 			#; % The robot system’s starting quantity of raw printing materials.
Env_Materials = 500.0 		#; % The environment’s quantity of collectable raw printing materials.
BaseCost_NonPr = 1 			#; % Base robot cost of nonprintable components.
PrintCost_NonPr = 1 		#; % Print capability cost of nonprintable components.
AssembleCost_NonPr = 1 		#; % Assemble capability cost of nonprintable components.
BaseCost_Pr = 2 			#; % Base robot cost of printable components.
PrintCost_Pr = 2 			#; % Print capability cost of printable components.
AssembleCost_Pr = 2 		#; % Assemble capability cost of printable components.
BaseCost_Time = 2 			#; % Base robot cost of build time (in time-steps).
PrintCost_Time = 2 			#; % Print capability cost of build time (in time-steps).
AssembleCost_Time = 2 		#; % Assemble capability cost of build time (in time-steps).
Print_Efficiency = 1.0 		#; % Factor that scales raw printing materials to printable components.
Print_Amount = 1.0 			#; % Amount of raw materials converted per print task.
Collect_Amount = 1.0 		#; % Raw printing materials per collecting robot per timestep.
QualityThreshold = 0.5 		#; % Robots with a quality below this are non-functional.
Quality_incr_Chance = 5.0 	#; % Chance that a new robot’s build quality will increase.
Quality_incr_Lower = 0.01 	#; % Lower bound for quality increase amount.
Quality_incr_Upper = 0.05 	#; % Upper bound for quality increase amount.
Quality_decr_Chance = 50.0 	#; % Chance that a new robot s build quality will decrease.
Quality_decr_Lower = 0.01 	#; % Lower bound for quality decrease amount.
Quality_decr_Upper = 0.25 	#; % Upper bound for quality decrease amount.
RiskAmount_Collect = 1.0 	#; % Risk chance for the collect task type.
RiskAmount_Assemble = 0.1 	#; % Risk chance for the assemble task type.
RiskAmount_Print = 0.1 		#; % Risk chance for the print task type.
RiskQuality_Modifier = 5.0 	#; % Multiplier for impact of quality defects on risk amount.
RiskFactory_Modifier = 0.1 	#; % Multiplier for impact of factory-made robots on risk amount

# [replicator,normal,assembler,printer]
cost_Pr = [6,2,4,4]			#; % Total cost printable
cost_NonPr = [3,1,2,2]		#; % Total cost nonprintable

timecost_base = 2			#; % time cost basic
timecost_print = 2			#; % time cost print capability
timecost_assemble = 2		#; % time cost assemble capability

timecost_normal = timecost_base
timecost_replicator = timecost_base+timecost_assemble+timecost_print
timecost_printer = timecost_base+timecost_print
timecost_assembler = timecost_base+timecost_assemble

timecost_repair_base = 1
timecost_repair_normal = timecost_repair_base
timecost_repair_replicator = timecost_repair_base
timecost_repair_printer = timecost_repair_base
timecost_repair_assembler = timecost_repair_base

table_columns = ["Time","NonPr","Printable","Materials","Env_Materials",
		"#Replicator","#Normal","#Assembler","#Printer",
		"#Assembling","#Printing","#Collecting","#Idle",
		"#In","#Out",
		"Average Build Quality in-service","Average Build Quality of System",
		"#WasteReplicator","#WasteNormal","#WasteAssembler","#WastePrinter",
		"Environment Exhaust Time", "Printable Exhaust Time", 
		"NonPr Exhaust Time","Material Exhaust Time","Average Risk"]


def resetGlobal(t,r,n,a,p):
    global rid,nid,aid,pid,Num_Steps,NonPr,Printable,Materials,Env_Materials
    
    # global variables
    rid = r
    nid = n
    aid = a
    pid = p

    # simulation parameters
    Num_Steps = 100				#; % Number of iterations/time-steps that the simulation goes through.
    NonPr = 300.0 				#; % The robot system=s starting quantity of nonprintable components.
    Printable = 100.0 			#; % The robot system’s starting quantity of printable components.
    Materials = 50.0 			#; % The robot system’s starting quantity of raw printing materials.
    Env_Materials = 500.0 		#; % The environment’s quantity of collectable raw printing materials.
    timesteps = t


# Object::Robot

In [90]:
# robot object
class Robot:
    def __init__(self,type,build_qual,id):
        self.type = type
        self.current_task = "idle"
        self.prev_task = "idle"
        self.id = self.type[0]+str(id)
        self.build_qual = round(build_qual,decimalPlaces)
        self.factory_made = True
        self.tasks_dur = 0
        self.taskindex = 0
        self.previouslyBuilt = ""
        self.prevTaskDur = 0
        self.curr_repair_task_dur = 0
        self.prev_repair_task_dur = 0
        self.factory = False
        self.risk = 0

        if(self.type == "Replicator"):
            self.tasks = ["Assemble","Print","Collect","Repair"]
            self.beingbuiltlist = []
        if(self.type == "Normal"):
            self.tasks = ["Collect","Repairing"]
        if(self.type == "Assembler"):
            self.tasks = ["Assemble","Collect","Repair"]
            self.beingbuiltlist = []
        if(self.type == "Printer"):
            self.tasks = ["Print","Collect","Repair"]
        self.num_tasks = len(self.tasks)

    def get_type(self):
        return self.type

    def __str__(self):
        return str(self.id)+" "+str(self.current_task)

    def set_curr_task(self,tasktype):
        self.current_task = tasktype
        if(self.current_task == "idle"):
            self.tasks_dur = 0
        if(self.current_task == "collecting"):
            self.tasks_dur = 1
        if(self.current_task == "assembling"):
            self.tasks_dur = 2
        if(self.current_task == "printing"):
            self.tasks_dur = 2
#         if(self.current_task == "repair"):
#             if(self.prev_task == "repair"):
#                 self.prev_repair_task_dur = self.curr_repair_task_dur
# #             self.tasks_dur = get_prev_task_dur() + timecost_repair_base
#             self.tasks_dur = get_prev_task_dur() + 1
#             self.curr_repair_task_dur = self.tasks_dur
        
    def setFactory(self):
        self.factory = True

    # methods of robot object
    def set_prev_task(self,tasktype):
        self.prev_task = tasktype
    def set_task_dur(self,task_dur):
        self.tasks_dur = task_dur
    def get_robot_id(self):
        return self.id
    def get_buid_qual(self):
        return self.build_qual
    def get_curr_task(self):
        return self.current_task
    def get_prev_task(self):
        return self.prev_task
    def get_task_dur(self):
        return self.tasks_dur
    def get_prev_task_dur(self):
        if(self.prev_task == "idle"):
            return 0
        if(self.prev_task == "collecting"):
            return 1
        if(self.prev_task == "assembling"):
            return 2
        if(self.prev_task == "printing"):
            return 2
        if(self.prev_task == "repair"):
            return self.prev_repair_task_dur 

    def set_previously_built(self, val):
        self.previouslyBuilt = val

    def get_previously_built(self):
        return self.previouslyBuilt
    
    def setRisk(self,risk):
        self.risk = risk
    def getRisk(self):
        return self.risk


# Functions

In [91]:

# print current resources
def printResources():
	print(NonPr,Printable,Materials,Env_Materials)

# check if robot can collect from Env_Materials
def collectCheck(robot):
	global Materials, Env_Materials, Collect_Amount
	if("Collect" in robot.tasks):
		if (Env_Materials - Collect_Amount >= 0):
			# Materials = Materials + Collect_Amount
			# Env_Materials = Env_Materials - Collect_Amount
			return True
		else:
			return False
	else: 
		return False

# task function - collecting
def collecting(robot):
    global Materials, Env_Materials, Collect_Amount
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("collecting")
    robot.set_task_dur(1)
    Materials = Materials + Collect_Amount
    Env_Materials = Env_Materials - Collect_Amount

# build robot task - assembler and replicator
# assemble task

def assembleCheck(robot,tobuild):
	# global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper
	if "Assemble" in robot.tasks:
		if(tobuild == "Replicator"):
			i=0
			# rid = rid+1
			# robotid = rid
		if(tobuild == "Normal"):
			i=1
			# nid = nid+1
			# robotid = nid
		if(tobuild == "Assembler"):
			i=2
			# aid = aid+1
			# robotid = aid
		if(tobuild == "Printer"):
			i=3
			# pid = pid+1
			# robotid = pid

		if Printable - cost_Pr[i] >= 0 and NonPr - cost_NonPr[i] >= 0:
			return True
		else:
			return False
	else:
		return False

def assembling(robot,tobuild):
	global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper
	
	if(tobuild == "Replicator"):
		i=0
		taskDur = timecost_replicator
	if(tobuild == "Normal"):
		i=1
		taskDur = timecost_normal
	if(tobuild == "Assembler"):
		i=2
		taskDur = timecost_assembler
	if(tobuild == "Printer"):
		i=3
		taskDur = timecost_printer
	
	robot.set_prev_task(robot.get_curr_task())
	robot.set_curr_task("assembling")
	robot.set_task_dur(taskDur)
	if(checkType(robot,"Assembler") or checkType(robot,"Replicator")):
		if(tobuild == "Replicator"):
			i=0
			rid = rid+1
			robotid = rid
		if(tobuild == "Normal"):
			i=1
			nid = nid+1
			robotid = nid
		if(tobuild == "Assembler"):
			i=2
			aid = aid+1
			robotid = aid
		if(tobuild == "Printer"):
			i=3
			pid = pid+1
			robotid = pid
	
		# subtract resources
		Printable = Printable - cost_Pr[i]
		NonPr = NonPr - cost_NonPr[i]
		robot.beingbuiltlist.append(tobuild[0]+str(robotid))
		robot.set_previously_built(tobuild)
		return True
	else:
		robot.set_prev_task(robot.get_curr_task())
		robot.set_curr_task("idle")
		robot.set_task_dur(0)
		return False

def assemble(builder,tobuild):
	global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper
	
	if(checkType(builder,"Assembler") or checkType(builder,"Replicator")):
		if(tobuild == "Replicator"):
			i=0
			
		if(tobuild == "Normal"):
			i=1
			
		if(tobuild == "Assembler"):
			i=2
			
		if(tobuild == "Printer"):
			i=3

		AssemblerQuality = builder.get_buid_qual()
		# robot's build quality		
		rand = round(random.uniform(0,1),decimalPlaces)
		if rand > round((1.0 - Quality_incr_Chance/100),decimalPlaces):
			RobotQuality = AssemblerQuality + random.uniform(Quality_incr_Lower, Quality_incr_Upper)
		elif rand < Quality_decr_Chance :
			RobotQuality = AssemblerQuality - random.uniform(Quality_decr_Lower, Quality_decr_Upper)
		else :
			RobotQuality = AssemblerQuality
		# print(builder,builder.beingbuiltlist)
		newRobot = Robot(tobuild,RobotQuality,builder.beingbuiltlist.pop(0)[1:])
		return newRobot
	else:
		return None


def printCheck(robot):
	global Print_Efficiency, Print_Amount, Materials, Printable
	if(checkType(robot,"Replicator") or checkType(robot,"Printer") and "Print" in robot.tasks):
		if Materials - (Print_Efficiency*Print_Amount) > 0:
			return True
		else:
			return False
	else:
		return False

def printing(robot):
    global Print_Efficiency, Print_Amount, Materials, Printable
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("printing")
    robot.set_task_dur(PrintCost_Time)
    Materials = Materials - (Print_Efficiency*Print_Amount)
    Printable = Printable + (Print_Efficiency*Print_Amount)

def resetTasks(robot):
    robot.set_prev_task(robot.get_curr_task())
    robot.set_task_dur(0)
    robot.set_curr_task("idle")

def checkCurrentTask(robot,current_task):
    return robot.get_curr_task() == current_task
def checkPreviousTask(robot,previous_task):
    return robot.get_prev_task() == previous_task

def checkType(robot,robot_type):
    return robot.get_type() == robot_type



# Task Risk

RiskTask = RiskTask_Type + (1.0 − RobotQuality) ∗ RiskTask_Type ∗ RiskQuality_Modifier

if Robot ∈ FactoryMade : RiskTask = RiskTask ∗ RiskFactory_Modifier

In [92]:
def taskRisk(robot):
    if robot.factory == True:
        currTask = robot.get_curr_task()
        if(currTask == "idle"):
            RiskTask_Type = 0
        elif(currTask == "collecting"):
            RiskTask_Type = 1
        elif(currTask == "assembling"):
            RiskTask_Type = 2
        elif(currTask == "printing"):
            RiskTask_Type = 2
        elif(currTask == "repair"):
            RiskTask_Type = 0
        riskTask = RiskTask_Type * RiskFactory_Modifier
    else:
        currTask = robot.get_curr_task()
        if(currTask == "idle"):
            RiskTask_Type = 0
            riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskTask_Type * RiskQuality_Modifier
        elif(currTask == "collecting"):
            RiskTask_Type = 1
            riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskTask_Type * RiskQuality_Modifier
        elif(currTask == "assembling"):
            RiskTask_Type = 2
            riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskTask_Type * RiskQuality_Modifier
        elif(currTask == "printing"):
            RiskTask_Type = 2
            riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskTask_Type * RiskQuality_Modifier
        elif(currTask == "repair"):
            RiskTask_Type = 0
            
        
    return riskTask

# Function::Configurations

In [93]:
def configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat):
        n_replicator = 0
        n_normal = 0
        n_assembler = 0
        n_printer = 0

        c_flag = 0
        p_flag = 0
        a_flag = 0
        i_flag = 0

        useless_c_flag = 0
        useless_p_flag = 0
        useless_a_flag = 0
        useless_r_flag = 0

        tot_build_qual_inservice = 0
        tot_build_qual_inoutservice = 0

        build_quality_list = []


        for i in totlist:
#             print(i.get_buid_qual())
            if(i.get_buid_qual()>=0.5):
                tot_build_qual_inoutservice = tot_build_qual_inoutservice + i.get_buid_qual()
                tot_build_qual_inservice = tot_build_qual_inservice + i.get_buid_qual()
            else:
                tot_build_qual_inoutservice = tot_build_qual_inoutservice + i.get_buid_qual()
            build_quality_list.append(i.get_buid_qual())
        
        avg_risk = 0
        for i in robotlist:
            if i.get_curr_task()=="collecting":
                c_flag+=1
            elif i.get_curr_task()=="printing":
                p_flag+=1
            elif i.get_curr_task()=="assembling":
                a_flag+=1
            elif i.get_curr_task()=="idle":
                # print(i)
                i_flag+=1

            if(checkType(i,"Replicator")):
                n_replicator += 1;
            elif(checkType(i, "Normal")):
                n_normal += 1;
            elif(checkType(i, "Assembler")):
                n_assembler += 1;
            elif(checkType(i, "Printer")):
                n_printer += 1;
                
            avg_risk += i.getRisk()
        avg_risk = avg_risk/len(robotlist)
        
        for i in useless:
            if(checkType(i,"Replicator")):
                useless_r_flag += 1;
            elif(checkType(i, "Normal")):
                useless_c_flag += 1;
            elif(checkType(i, "Assembler")):
                useless_a_flag += 1;
            elif(checkType(i, "Printer")):
                useless_p_flag += 1;



        avg_build_qual_inservice = round(tot_build_qual_inservice/len(robotlist),decimalPlaces)
        avg_build_qual_inoutservice = round(tot_build_qual_inoutservice/len(totlist),decimalPlaces)
        
#         print(tot_build_qual_inservice,tot_build_qual_inoutservice)
        
#         print(avg_build_qual_inservice,avg_build_qual_inoutservice)

#         listnumCollecting.append(c_flag)
#         listnumPrinting.append(p_flag)
#         listnumAssembling.append(a_flag)


        ids=[]
        for j in totlist:
            isWaste = False
            if j.build_qual<=QualityThreshold:
                isWaste = True
            ids.append(j.id)

        if (Env_Materials == 0 and checkENV == 0):
            checkENV = t
        if (Printable == 0 and checkPrint == 0):
            checkPrint = t
        if (NonPr == 0 and checkNonPr == 0):
            checkNonPr = t
        if (Materials <= 1 and checkMat == 0):
            checkMat = t
            
# 		"#Replicator","#Normal","#Assembler","#Printer",
# 		"#Assembling","#Printing","#Collecting","#Idle",
# 		"#In","#Out",
# 		"Average Build Quality in-service","Average Build Quality of System",
# 		"#WasteReplicator","#WasteNormal","#WasteAssembler","#WastePrinter",
# 		"Environment Exhaust Time", "Printable Exhaust Time", 
# 		"NonPr Exhaust Time","Material Exhaust Time"]

        
        return build_quality_list,[n_replicator,n_normal,n_assembler,n_printer,
        a_flag,p_flag,c_flag,i_flag,
        len(robotlist),len(useless),
        avg_build_qual_inservice,avg_build_qual_inoutservice,
        useless_r_flag,useless_c_flag,useless_a_flag,useless_p_flag,
        checkENV,checkPrint,checkNonPr,checkMat,avg_risk]

In [96]:
def CHO(timesteps,robotlist,df):
    totlist = robotlist
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0,timesteps):
        print("="*20)
        for i in range(len(robotlist)):
            
            print(t,i,taskRisk(robotlist[i]),robotlist[i])
            
            # IDLE
            if(checkCurrentTask(robotlist[i],"idle")):
#                 robotlist[i].setRisk(taskRisk(robotlist[i]))
                if(checkType(robotlist[i],"Replicator")):
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])	
                    else:
                        resetTasks(robotlist[i])
                elif(checkType(robotlist[i],"Normal")):
                    if (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        resetTasks(robotlist[i])

            # NOT IDLE
            else:
#                 robotlist[i].setRisk(taskRisk(robotlist[i]))
#                 print(t,taskRisk(robotlist[i]),robotlist[i])
#                 robotlist[i].setRisk(taskRisk(robotlist[i]))
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and checkType(robotlist[i],"Replicator")):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Normal")):
                        assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        isAssembling = False
                        resetTasks(robotlist[i])

                    # it enters this loop only when it has to pop a new robot
                    if(checkPreviousTask(robotlist[i],"assembling")):
                        newbot = assemble(robotlist[i],"Normal")
                        if newbot and newbot.build_qual>=0.5:
                            if(checkType(newbot,"Normal")):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                            resetTasks(robotlist[i])
                        # robotlist[i].set_prev_task(robotlist[i].current_task)

                # Normal
                elif(checkType(robotlist[i],"Normal")):
                    canCollect = collectCheck(robotlist[i])
                    if(canCollect):
                        collecting(robotlist[i])
                    else:
                        resetTasks(robotlist[i])

        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+(vals)
        
        
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
        
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df

# Testing

In [97]:
def main(configname,timesteps):
    build_qual_range = [0.85,0.95]
    init_build_qual = random.uniform(build_qual_range[0],build_qual_range[1])
    df = pd.DataFrame(columns = table_columns)
    
    if configname=="CHO":
        robot = Robot("Replicator",init_build_qual,rid)
        robot.setFactory()
        robotlist = [robot]
        resetGlobal(timesteps,1,0,0,0)
        df = CHO(timesteps,robotlist,df)  
#         plotdf(df,configname,["#In", "#Out"])
    
    display(df)
        
if __name__ == "__main__":
    main("CHO",75) #2000

0 0 0.0 R1 idle
1 0 0.2 R1 assembling
2 0 0.2 R1 assembling
3 0 0.2 R1 assembling
3 1 2.5799999999999996 N1 collecting
3 2 2.5799999999999996 N1 collecting
4 0 0.2 R1 assembling
4 1 2.5799999999999996 N1 collecting
4 2 2.5799999999999996 N1 collecting
5 0 0.2 R1 assembling
5 1 2.5799999999999996 N1 collecting
5 2 2.5799999999999996 N1 collecting
5 3 2.125 N2 collecting
5 4 2.125 N2 collecting
6 0 0.2 R1 assembling
6 1 2.5799999999999996 N1 collecting
6 2 2.5799999999999996 N1 collecting
6 3 2.125 N2 collecting
6 4 2.125 N2 collecting
7 0 0.2 R1 assembling
7 1 2.5799999999999996 N1 collecting
7 2 2.5799999999999996 N1 collecting
7 3 2.125 N2 collecting
7 4 2.125 N2 collecting
7 5 1.87 N3 collecting
7 6 1.87 N3 collecting
8 0 0.2 R1 assembling
8 1 2.5799999999999996 N1 collecting
8 2 2.5799999999999996 N1 collecting
8 3 2.125 N2 collecting
8 4 2.125 N2 collecting
8 5 1.87 N3 collecting
8 6 1.87 N3 collecting
9 0 0.2 R1 assembling
9 1 2.5799999999999996 N1 collecting
9 2 2.579999999999999

,Time,NonPr,Printable,Materials,Env_Materials,#Replicator,#Normal,#Assembler,#Printer,#Assembling,#Printing,#Collecting,#Idle,#In,#Out,Average Build Quality in-service,Average Build Quality of System,#WasteReplicator,#WasteNormal,#WasteAssembler,#WastePrinter,Environment Exhaust Time,Printable Exhaust Time,NonPr Exhaust Time,Material Exhaust Time,Average Risk,Print Capacity,Assembling Capacity,Collection Capacity
0,0.0,299.0,98.0,50.0,500.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.909,0.909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,1.0,299.0,98.0,50.0,500.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.909,0.909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,2.0,298.0,96.0,51.0,499.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.759,0.759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0
3,3.0,298.0,96.0,53.0,497.0,1.0,2.0,0.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.759,0.759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0
4,4.0,297.0,94.0,56.0,494.0,1.0,4.0,0.0,0.0,1.0,0.0,4.0,0.0,5.0,0.0,0.765,0.765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,70.0,264.0,28.0,550.0,0.0,1.0,70.0,0.0,0.0,1.0,0.0,0.0,70.0,71.0,0.0,0.783,0.783,0.0,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,1.0,1.0,71.0
71,71.0,264.0,28.0,550.0,0.0,1.0,70.0,0.0,0.0,1.0,0.0,0.0,70.0,71.0,0.0,0.783,0.783,0.0,0.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,1.0,1.0,71.0
72,72.0,263.0,26.0,550.0,0.0,1.0,72.0,0.0,0.0,1.0,0.0,0.0,72.0,73.0,0.0,0.780,0.780,0.0,0.0,0.0,0.0,72.0,0.0,0.0,0.0,0.0,1.0,1.0,73.0
73,73.0,263.0,26.0,550.0,0.0,1.0,72.0,0.0,0.0,1.0,0.0,0.0,72.0,73.0,0.0,0.780,0.780,0.0,0.0,0.0,0.0,73.0,0.0,0.0,0.0,0.0,1.0,1.0,73.0


# Do not run below

In [1619]:
def DHO(timesteps,robotlist,df):
    totlist = robotlist
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0,timesteps):

        for i in range(len(robotlist)):
            # IDLE
            if(robotlist[i].current_task=="idle"):

                # Replicator
                if(robotlist[i].type == "Replicator"):

                    if(assembleCheck(robotlist[i],"Replicator")):
                        assembling(robotlist[i],"Replicator")
                    elif(printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])	
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    # print(t,robotlist[i].id,canCollect)
                    if canCollect:
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Replicator")):
                        isAssembling = assembling(robotlist[i],"Replicator")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])	
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])	
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    # it enters this loop only when it has to pop a new robot
                    if(robotlist[i].get_prev_task()=="assembling"):
                        newbot = assemble(robotlist[i],"Replicator")
                        if newbot and newbot.build_qual>=0.5:
                            if(newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            if(newbot.type == "Replicator"):
                                if(assembleCheck(newbot,"Replicator")):
                                    assembling(newbot,"Replicator")
                                elif(printCheck(robotlist[i])):
                                    printing(robotlist[i])
                                elif(collectCheck(robotlist[i])):
                                    collecting(robotlist[i])	
                                else:
                                    newbot.set_prev_task(robotlist[i].get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)

                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if(canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
        
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df

In [1620]:
def HHO(timesteps,robotlist,df):
    totlist = robotlist
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0, timesteps):

        for i in range(len(robotlist)):
            # IDLE
            if (robotlist[i].current_task == "idle"):
                # Replicator
                if (robotlist[i].type == "Replicator"):

                    if (robotlist[i].get_previously_built() == ""):
                        if (assembleCheck(robotlist[i],"Replicator")):
                            isAssembling = assembling(robotlist[i], "Replicator")
                        elif (assembleCheck(robotlist[i],"Normal")):
                            isAssembling = assembling(robotlist[i], "Normal")
                        elif (printCheck(robotlist[i])):
                            printing(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")

                    else:
                        if (assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Replicator"):
                            assembling(robotlist[i], "Normal")
                        elif (assembleCheck(robotlist[i],"Replicator") and robotlist[i].get_previously_built() == "Normal"):
                            assembling(robotlist[i], "Replicator")
                        elif (printCheck(robotlist[i])):
                            printing(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")

                # Normal
                elif (robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if canCollect:
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

            # NOT IDLE
            else:
                # reduce task duration every time step
                if (robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                # Replicator
                elif (robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if (assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Replicator"):
                        assembling(robotlist[i], "Normal")
                    elif (assembleCheck(robotlist[i],"Replicator") and robotlist[i].get_previously_built() == "Normal"):
                        assembling(robotlist[i], "Replicator")
                        # checking if robot can collect
                    elif (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    # it enters this loop only when it has to pop a new robot
                    if (robotlist[i].get_prev_task() == "assembling"):

                        if (robotlist[i].get_previously_built() == "Normal"):
                            newbot = assemble(robotlist[i], "Replicator")
                        elif (robotlist[i].get_previously_built() == "Replicator"):
                            newbot = assemble(robotlist[i], "Normal")
                        else:
                            newbot = assemble(robotlist[i], "Replicator")

                        if newbot and newbot.build_qual >= 0.5:

                            if (newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)

                            if (newbot.type == "Replicator"):

                                if (newbot.get_previously_built() == ""):
                                    if (assembleCheck(newbot,"Replicator")):
                                        isAssembling = assembling(newbot, "Replicator")
                                    elif (assembleCheck(newbot,"Normal")):
                                        isAssembling = assembling(newbot, "Normal")
                                    elif (printCheck(newbot)):
                                        printing(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")

                                else:
                                    if (assembleCheck(newbot,"Normal") and newbot.get_previously_built() == "Replicator"):
                                        assembling(newbot, "Normal")
                                    elif (assembleCheck(newbot,"Replicator") and newbot.get_previously_built() == "Normal"):
                                        assembling(newbot, "Replicator")
                                    # checking if robot can collect
                                    elif (printCheck(newbot)):
                                        printing(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")


                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
#                         robotlist[i].set_prev_task(robotlist[i].current_task)

                # Normal
                elif (robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if (canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")


                        
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
        
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df

In [1621]:
def CHE(timesteps,robotlist,df):
    totlist = robotlist
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0,timesteps):

        #Parsing through complete robot list to check their availability
        for i in range(len(robotlist)):
            #Checking if robot is idle
            if(robotlist[i].current_task=="idle"):

                #If idle and replicator
                if(robotlist[i].type == "Replicator"):

                    #Checking if robot can assemble
                    if(assembleCheck(robotlist[i],"Replicator")):
                        #Starting the assembly process + reducing resources
                        assembling(robotlist[i],"Replicator")

                    #checking if robot can print
                    elif(printCheck(robotlist[i])):
                        #Starting the printing process + reducing resources
                        printing(robotlist[i])

                    #checking if robot can collect
                    elif(collectCheck(robotlist[i])):
                        #Starting the collecting process + reducing resources
                        collecting(robotlist[i])

                    #If can't do any task then set robot to idle
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                # If idle and printer
                elif(robotlist[i].type == "Printer"):
                    if(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                # If idle and assembler
                elif(robotlist[i].type == "Assembler"):

                    if(assembleCheck(robotlist[i],"Normal")):
                        assembling(robotlist[i],"Normal")
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                #If idle and collector
                elif(robotlist[i].type == "Normal"):
                    # checking if robot can collect
                    if (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

            #If robot is not idle
            else:

                #Reduce task duration if task not ending in the next time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                #If task is ending in the next time step and Printer
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Printer"):

                    # checking if robot can print
                    if(printCheck(robotlist[i])):
                        # Starting the printing process + reducing resources
                        printing(robotlist[i])

                    # checking if robot can collect
                    elif(collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])

                    # If can't do any task then set robot to idle
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")




                #If task is ending in the next time step and current robot is Assembler
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Assembler"):

                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Normal")):
                        # Starting the assembly process + reducing resources
                        assembling(robotlist[i],"Normal")

                    # checking if robot can collect
                    elif(collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])

                    # If can't do any task then set robot to idle
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    #If robot task is ending in the next step
                    #Create a new robot for the robot list that can start working the next cycle
                    if(robotlist[i].get_prev_task()=="assembling"):

                        # Build a new replicator
                        newbot = assemble(robotlist[i],"Normal")

                        # If newbot passes the quality check
                        if newbot and newbot.build_qual>=0.5:

                            #If newbot is a collector
                            if(newbot.type == "Normal"):
                                # checking if robot can collect
                                if(collectCheck(newbot)):
                                    # Starting the collecting process + reducing resources
                                    collecting(newbot)
                                else:
                                    newbot.set_prev_task(newbot.get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")

                            #Adding the newbot to the total robot list and robot list
                            totlist.append(newbot)
                            robotlist.append(newbot)

                        # If newbot does not pass the quality check
                        else:
                            # Adding the newbot to the total robot list and useless list
                            totlist.append(newbot)
                            useless.append(newbot)

                #If task is ending in the next time step and Collector
                elif(robotlist[i].type == "Normal"):

                    # checking if robot can collect
                    if(collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        # set current task to idle if can not collect
                        robotlist[i].set_curr_task("idle")



                        
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
        
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df

In [1622]:
def DHE(timesteps,robotlist,df):
    totlist = robotlist
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0, timesteps):

        # Parsing through complete robot list to check their availability
        for i in range(len(robotlist)):
            # Checking if robot is idle
            if (robotlist[i].current_task == "idle"):

                # If idle and printer
                if (robotlist[i].type == "Printer"):

                    if (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                # If idle and assembler
                elif (robotlist[i].type == "Assembler"):

                    if (robotlist[i].get_previously_built() == ""):
                        if (assembleCheck(robotlist[i],"Assembler")):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (assembleCheck(robotlist[i],"Printer")):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")
                    else:
                        if (assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                            assembling(robotlist[i], "Printer")
                        elif (assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Printer"):
                            assembling(robotlist[i], "Assembler")
                        # checking if robot can collect
                        elif (collectCheck(robotlist[i])):
                            # Starting the collecting process + reducing resources
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")


            # If robot is not idle
            else:

                # Reduce task duration if task not ending in the next time step
                if (robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                # If task is ending in the next time step and Printer
                elif (robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Printer"):

                    if (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")


                # If task is ending in the next time step and current robot is Assembler
                elif (robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Assembler"):

                    if (assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                        assembling(robotlist[i], "Printer")
                    elif (assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Printer"):
                        assembling(robotlist[i], "Assembler")
                        # checking if robot can collect
                    elif (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    # If robot task is ending in the next step
                    # Create a new robot for the robot list that can start working the next cycle
                    if (robotlist[i].get_prev_task() == "assembling"):

                        # Build a new robot in a fixed pattern
                        if (robotlist[i].get_previously_built() == "Printer"):
                            newbot = assemble(robotlist[i], "Assembler")
                        elif (robotlist[i].get_previously_built() == "Assembler"):
                            newbot = assemble(robotlist[i], "Printer")
                        else:
                            newbot = assemble(robotlist[i], "Assembler")

                        # Build a new replicator
                        # newbot = assemble(robotlist[i], robotlist[i].get_previously_built())
                        # If newbot passes the quality check
                        if newbot and newbot.build_qual >= 0.5:

                            # If newbot is a collector))
                            if (newbot.type == "Printer"):
                                if (printCheck(newbot)):
                                    printing(newbot)
                                elif (collectCheck(newbot)):
                                    collecting(newbot)
                                else:
                                    newbot.set_prev_task(newbot.get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")
                            # If idle and assembler
                            elif (newbot.type == "Assembler"):
                                if (newbot.get_previously_built() == ""):
                                    if (assembleCheck(newbot,"Assembler")):
                                        isAssembling = assembling(newbot, "Assembler")
                                    elif (assembleCheck(newbot,"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")
                                else:
                                    if (newbot[i].get_previously_built() == "Assembler" and assembleCheck(newbot,"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (newbot[i].get_previously_built() == "Printer" and assembleCheck(newbot,"Assembler")):
                                        isAssembling = assembling(newbot, "Assembler")
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        newbot.set_prev_task(newbot.get_curr_task())
                                        newbot.set_task_dur(0)
                                        newbot.set_curr_task("idle")

                            # Adding the newbot to the total robot list and robot list
                            totlist.append(newbot)
                            robotlist.append(newbot)

                        # If newbot does not pass the quality check
                        else:
                            # Adding the newbot to the total robot list and useless list
                            totlist.append(newbot)
                            useless.append(newbot)

                    # check if it can keep assembling next time step


                # If task is ending in the next time step and Collector
                elif (robotlist[i].type == "Normal"):

                    # checking if robot can collect
                    if (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        # set current task to idle if can not collect
                        robotlist[i].set_curr_task("idle")
                        
                        
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
        
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df


In [1623]:
def HHE(timesteps,robotlist,df):
    totlist = robotlist
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    # For loop for each time step
    for t in range(0, timesteps):

        # Parsing through complete robot list to check their availability
        for i in range(len(robotlist)):
            # Checking if robot is idle
            if (robotlist[i].current_task == "idle"):

                # If idle and replicator
                if (robotlist[i].type == "Replicator"):

                    # Checking if robot can assemble
                    if (assembleCheck(robotlist[i],"Replicator")):
                        # Starting the assembly process + reducing resources
                        assembling(robotlist[i], "Replicator")

                    # checking if robot can print
                    elif (printCheck(robotlist[i])):
                        # Starting the printing process + reducing resources
                        printing(robotlist[i])

                    # checking if robot can collect
                    elif (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])

                    # If can't do any task then set robot to idle
                    else:
                        resetTasks(robotlist[i])

                # If idle and printer
                elif (robotlist[i].type == "Printer"):

                    if (printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        resetTasks(robotlist[i])
                        
                # If idle and assembler
                elif (robotlist[i].type == "Assembler"):

                    if (robotlist[i].get_previously_built() == ""):
                        if (assembleCheck(robotlist[i],"Normal")):
                            isAssembling = assembling(robotlist[i], "Normal")
                        elif (assembleCheck(robotlist[i],"Assembler")):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (assembleCheck(robotlist[i],"Printer")):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                            robotlist[i].set_task_dur(0)
                            robotlist[i].set_curr_task("idle")
                    else:
                        if (robotlist[i].get_previously_built() == "Assembler" and assembleCheck(robotlist[i],"Printer")):
                            isAssembling = assembling(robotlist[i], "Printer")
                        elif (robotlist[i].get_previously_built() == "Printer" and assembleCheck(robotlist[i],"Normal")):
                            isAssembling = assembling(robotlist[i], "Normal")
                        elif (robotlist[i].get_previously_built() == "Normal" and assembleCheck(robotlist[i],"Assembler")):
                            isAssembling = assembling(robotlist[i], "Assembler")
                        elif (collectCheck(robotlist[i])):
                            collecting(robotlist[i])
                        else:
                            resetTasks(robotlist[i])

                # If idle and collector
                elif (robotlist[i].type == "Normal"):
                    # checking if robot can collect
                    if (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])
                    else:
                        resetTasks(robotlist[i])

            # If robot is not idle
            else:

                # Reduce task duration if task not ending in the next time step
                if (robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                # If task is ending in the next time step and Printer
                elif (robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Printer"):

                    # checking if robot can print
                    if (printCheck(robotlist[i])):
                        # Starting the printing process + reducing resources
                        printing(robotlist[i])

                    # checking if robot can collect
                    elif (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])

                    # If can't do any task then set robot to idle
                    else:
                        resetTasks(robotlist[i])


                # If task is ending in the next time step and current robot is Assembler
                elif (robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Assembler"):

                    if (assembleCheck(robotlist[i],"Normal") and robotlist[i].get_previously_built() == "Printer"):
                        assembling(robotlist[i], "Normal")
                    elif (assembleCheck(robotlist[i],"Printer") and robotlist[i].get_previously_built() == "Assembler"):
                        assembling(robotlist[i], "Printer")
                    elif (assembleCheck(robotlist[i],"Assembler") and robotlist[i].get_previously_built() == "Normal"):
                        assembling(robotlist[i], "Assembler")
                        # checking if robot can collect
                    elif (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])

                        # If can't do any task then set robot to idle
                    else:
                        resetTasks(robotlist[i])

                    # If robot task is ending in the next step
                    # Create a new robot for the robot list that can start working the next cycle
                    if (robotlist[i].get_prev_task() == "assembling"):

                        # Build a new robot in a fixed pattern
                        if (robotlist[i].get_previously_built() == ""):
                            newbot = assemble(robotlist[i], "Normal")
                        elif (robotlist[i].get_previously_built() == "Normal"):
                            newbot = assemble(robotlist[i], "Printer")
                        elif (robotlist[i].get_previously_built() == "Printer"):
                            newbot = assemble(robotlist[i], "Assembler")
                        elif (robotlist[i].get_previously_built() == "Assembler"):
                            newbot = assemble(robotlist[i], "Normal")
                        else:
                            newbot = assemble(robotlist[i], "Normal")

                        # Build a new replicator
                        # newbot = assemble(robotlist[i], robotlist[i].get_previously_built())
                        # If newbot passes the quality check
                        if newbot and newbot.build_qual >= 0.5:

                            # If newbot is a collector))
                            if (checkType(newbot,"Normal")):
                                # checking if robot can collect
                                if (collectCheck(newbot)):
                                    # Starting the collecting process + reducing resources
                                    collecting(newbot)
                                else:
                                    resetTasks(newbot)

                            elif (newbot.type == "Printer"):
                                if (printCheck(newbot)):
                                    printing(newbot)
                                elif (collectCheck(newbot)):
                                    collecting(newbot)
                                else:
                                    newbot.set_prev_task(newbot.get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")
                            # If idle and assembler
                            elif (checkType(newbot,"Assembler")):
                                if (newbot.get_previously_built() == ""):
                                    if (assembleCheck(newbot,"Normal")):
                                        isAssembling = assembling(newbot, "Normal")
                                    elif (assembleCheck(newbot,"Assembler")):
                                        isAssembling = assembling(newbot, "Assembler")
                                    elif (assembleCheck(newbot,"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        resetTasks(newbot)
                                else:
                                    if (newbot[i].get_previously_built() == "Assembler" and assembleCheck(newbot,"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (newbot[i].get_previously_built() == "Printer" and assembleCheck(newbot,"Normal")):
                                        isAssembling = assembling(newbot, "Normal")
                                    elif (newbot[i].get_previously_built() == "Normal" and assembleCheck(newbot,"Printer")):
                                        isAssembling = assembling(newbot, "Printer")
                                    elif (collectCheck(newbot)):
                                        collecting(newbot)
                                    else:
                                        resetTasks(newbot)
                            # Adding the newbot to the total robot list and robot list
                            totlist.append(newbot)
                            robotlist.append(newbot)

                        # If newbot does not pass the quality check
                        else:
                            # Adding the newbot to the total robot list and useless list
                            totlist.append(newbot)
                            useless.append(newbot)

                    # check if it can keep assembling next time step


                # If task is ending in the next time step and Collector
                elif (robotlist[i].type == "Normal"):

                    # checking if robot can collect
                    if (collectCheck(robotlist[i])):
                        # Starting the collecting process + reducing resources
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        # set current task to idle if can not collect
                        robotlist[i].set_curr_task("idle")
        
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
        
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df

In [1624]:
def plotdf(df,configname,cols):
    fig = px.bar(df, x="Time", y=cols, title=configname+ " - Habitat over Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show()
   
        
#     ['Time', 'NonPr', 'Printable', 'Materials', 'Env_Materials',
#        '#Replicator', '#Normal', '#Assembler', '#Printer', '#Assembling',
#        '#Printing', '#Collecting', '#Idle', '#In', '#Out',
#        'Average Build Quality in-service', 'Average Build Quality of System',
#        '#WasteReplicator', '#WasteNormal', '#WasteAssembler', '#WastePrinter',
#        'Environment Exhaust Time', 'Printable Exhaust Time',
#        'NonPr Exhaust Time', 'Material Exhaust Time', 'Print Capacity',
#        'Assembling Capacity', 'Collection Capacity']

#     pio.write_html(fig, file=configname+' - Habitat over time.html', auto_open=False, include_plotlyjs='cdn')
    fig.write_html(configname+' - Habitat over time.html', include_plotlyjs='cdn')
    cspy.iplot(fig,filename=configname+' - Habitat over time.html',auto_open=False)

    fig = make_subplots(rows=2, cols=2,
        subplot_titles=("Types of Robots vs Time", "Resources vs Time",
                        "Tasks of Robots vs Time", "Build Quality vs Time"),
        specs=[[{"type": "bar"}, {"type": "scatter"}],[{"type": "bar"}, {"type": "scatter"}]])
    
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Replicator'],name='#Replicator'),row=1, col=1)
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Normal'],name='#Normal'),row=1, col=1)
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Assembler'],name='#Assembler'),row=1, col=1)
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Printer'],name='#Printer'),row=1, col=1)
    

    fig.add_trace(go.Bar(x=df['Time'],y=df['#Assembling'],name='#Assembling'),row=2, col=1)
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Printing'],name='#Printing'),row=2, col=1)
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Collecting'],name='#Collecting'),row=2, col=1)
    fig.add_trace(go.Bar(x=df['Time'],y=df['#Idle'],name='#Idle'),row=2, col=1)
    
    fig.add_trace(go.Scatter(x=df["Time"], y=df['NonPr'], mode="lines",name='NonPr'),row=1, col=2)
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Printable'], mode="lines",name='Printable'),row=1, col=2)
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Materials'], mode="lines",name='Materials'),row=1, col=2)
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Env_Materials'], mode="lines",name='Env_Materials'),row=1, col=2)

    fig.add_trace(go.Scatter(x=df["Time"], y=df['Average Build Quality in-service'], mode="lines",name='Avg. Quality #in'),row=2, col=2)#                               
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Average Build Quality of System'], mode="lines",name='Avg. Quality #total'),row=2, col=2)#                               
    fig.update_layout(hovermode="x",title=configname+" - Characteristics")
#     fig.update_layout(height=700, showlegend=False)

    fig.show()
    fig.write_html(configname+' - Characteristics.html', include_plotlyjs='cdn')
    cspy.iplot(fig,filename=configname+' - Characteristics.html',auto_open=False)
#     pio.write_html(fig, file=configname+' - Characteristics.html', auto_open=False, include_plotlyjs='cdn')
    

# main( )

In [14]:
def main(configname,timesteps):
    build_qual_range = [0.85,0.95]
    init_build_qual = random.uniform(build_qual_range[0],build_qual_range[1])
    df = pd.DataFrame(columns = table_columns)
    
    if configname=="CHO":
        robot = Robot("Replicator",init_build_qual,rid)
        robotlist = [robot]
        resetGlobal(timesteps,1,0,0,0)
        df = CHO(timesteps,robotlist,df)  
        plotdf(df,configname,["#In", "#Out"])
        
    elif configname=="DHO":
        robot = Robot("Replicator",init_build_qual,rid)
        robotlist = [robot]
        resetGlobal(timesteps,1,0,0,0)
        df = DHO(timesteps,robotlist,df)  
        plotdf(df,configname,["#In", "#Out"])
        
    elif configname=="HHO":
        robot = Robot("Replicator",init_build_qual,rid)
        robotlist = [robot]
        resetGlobal(timesteps,1,0,0,0)
        df = HHO(timesteps,robotlist,df)  
        plotdf(df,configname,["#In", "#Out"])
        
    elif configname=="CHE":
        robot1 = Robot("Printer",init_build_qual,pid)
        robot2 = Robot("Assembler",init_build_qual,aid)
        robotlist = [robot1,robot2]
        resetGlobal(timesteps,0,0,1,1)
        df = CHE(timesteps,robotlist,df)  
        plotdf(df,configname,["#In", "#Out"])
        
    elif configname=="DHE":
        robot1 = Robot("Printer",init_build_qual,pid)
        robot2 = Robot("Assembler",init_build_qual,aid)
        robotlist = [robot1,robot2]
        resetGlobal(timesteps,0,0,1,1)
        df = DHE(timesteps,robotlist,df)  
        plotdf(df,configname,["#In", "#Out"])
        
    elif configname=="HHE":
        robot1 = Robot("Printer",init_build_qual,pid)
        robot2 = Robot("Assembler",init_build_qual,aid)
        robotlist = [robot1,robot2]
        resetGlobal(timesteps,0,0,1,1)
        df = HHE(timesteps,robotlist,df)  
        plotdf(df,configname,["#In", "#Out"])
#         display(df.head())
        
#         fig = px.bar(df, x="Time", y=["#In", "#Out"], title=configname,labels={'#In':'#in-service','#Out':'#out-service'})
#         fig.show()


# Run

In [1626]:
if __name__ == "__main__":
    main("CHO",75) #2000
    main("DHO",75)
    main("HHO",75)
    main("CHE",75) #1500
    main("DHE",75)
    main("HHE",75)